Q-2. Imagine you have a dataset where you have different features like Age ,

Gender , Height , Weight , BMI , and Blood Pressure and you have to classify the people into
different classes like Normal , Overweight , Obesity , Underweight , and Extreme Obesity by using
any 4 different classification algorithms. Now you have to build a model which
can classify people into different classes.
Dataset This is the Dataset You can use this dataset for this question.

In [27]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as  plt
import seaborn as sns

from sklearn.model_selection import train_test_split,cross_validate
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.svm import SVC
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

In [2]:
df=pd.read_csv(r"ObesityDataSet_raw_and_data_sinthetic.csv")
df.head()

,Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,NObeyesdad
0,Female,21.0,1.62,64.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,0.0,1.0,no,Public_Transportation,Normal_Weight
1,Female,21.0,1.52,56.0,yes,no,3.0,3.0,Sometimes,yes,3.0,yes,3.0,0.0,Sometimes,Public_Transportation,Normal_Weight
2,Male,23.0,1.80,77.0,yes,no,2.0,3.0,Sometimes,no,2.0,no,2.0,1.0,Frequently,Public_Transportation,Normal_Weight
3,Male,27.0,1.80,87.0,no,no,3.0,3.0,Sometimes,no,2.0,no,2.0,0.0,Frequently,Walking,Overweight_Level_I
4,Male,22.0,1.78,89.8,no,no,2.0,1.0,Sometimes,no,2.0,no,0.0,0.0,Sometimes,Public_Transportation,Overweight_Level_II


In [3]:
df.isnull().sum()

Gender                            0
Age                               0
Height                            0
Weight                            0
family_history_with_overweight    0
FAVC                              0
FCVC                              0
NCP                               0
CAEC                              0
SMOKE                             0
CH2O                              0
SCC                               0
FAF                               0
TUE                               0
CALC                              0
MTRANS                            0
NObeyesdad                        0
dtype: int64

In [6]:
#Renaming it
df = df.rename(columns={'family_history_with_overweight': 'FHWO',
                           'NObeyesdad' : 'BMI',})

In [8]:
#Lets see all the object related datatype
object_cols = [col for col in df.columns if df[col].dtype == "object"]

good_label_cols = [col for col in object_cols if set(df[col]).issubset(set(df[col]))]
good_label_cols.remove('BMI')

print('Categorical columns that will be ordinal encoded:', good_label_cols)

Categorical columns that will be ordinal encoded: ['Gender', 'FHWO', 'FAVC', 'CAEC', 'SMOKE', 'SCC', 'CALC', 'MTRANS']


In [9]:
data_transform = df.copy()
ordinal_encoder = OrdinalEncoder()
data_transform[good_label_cols] = ordinal_encoder.fit_transform(data_transform[good_label_cols])

In [10]:
dic_to_replace = {"BMI": {"Insufficient_Weight": 0, 
                          "Normal_Weight": 1,
                         'Overweight_Level_I': 2,
                          'Overweight_Level_II': 3,
                          'Obesity_Type_I': 4,
                          'Obesity_Type_II': 5,
                          'Obesity_Type_III': 6,}
                         }
data_transform.replace(dic_to_replace, inplace=True)

In [11]:
data_input = df.copy()
data_input = data_input.drop(["Gender",'NCP','SMOKE','MTRANS'],axis=1)

In [12]:
#Remove the unnecessary columns
good_label_cols.remove('Gender')
good_label_cols.remove('SMOKE')
good_label_cols.remove('MTRANS')

In [13]:
data_grouped_by = data_input.groupby(['BMI'])
data_balanced = data_grouped_by.apply(lambda x: x.sample(data_grouped_by.size().min()).reset_index(drop=True))
data_bal = data_balanced.droplevel(['BMI'])
data_bal

,Age,Height,Weight,FHWO,FAVC,FCVC,CAEC,CH2O,SCC,FAF,TUE,CALC,BMI
0,21.084625,1.787264,58.585146,yes,yes,2.530233,Sometimes,2.387945,no,1.976341,1.672508,no,Insufficient_Weight
1,18.006742,1.700000,50.000000,no,yes,1.003566,Frequently,1.014634,no,0.783676,1.000000,Sometimes,Insufficient_Weight
2,19.317148,1.731195,49.650897,no,yes,2.919751,Sometimes,1.949308,no,1.940182,1.000000,Sometimes,Insufficient_Weight
3,19.000000,1.530875,42.000000,no,no,2.844607,Frequently,1.695510,no,0.260079,0.635867,Sometimes,Insufficient_Weight
4,21.798856,1.672007,49.980968,yes,yes,2.979383,Frequently,2.975887,no,0.945093,1.241755,no,Insufficient_Weight
...,...,...,...,...,...,...,...,...,...,...,...,...,...
267,28.167799,1.658202,78.000000,yes,yes,2.938031,Sometimes,1.931242,no,0.525002,0.371941,Frequently,Overweight_Level_II
268,18.000000,1.751278,86.963628,yes,no,3.000000,Sometimes,2.000000,no,1.271667,0.155278,Sometimes,Overweight_Level_II
269,23.000000,1.715118,81.650778,yes,yes,2.000000,Sometimes,1.628997,no,0.880584,1.127675,Sometimes,Overweight_Level_II
270,19.703095,1.704141,78.790936,yes,no,1.650505,Sometimes,2.000000,no,1.732340,1.000000,Sometimes,Overweight_Level_II


In [15]:
data_preprocess = pd.get_dummies(data_bal, columns = good_label_cols)
dic_to_replace = {"BMI": {"Insufficient_Weight": 0, 
                          "Normal_Weight": 1,
                         'Overweight_Level_I': 2,
                          'Overweight_Level_II': 3,
                          'Obesity_Type_I': 4,
                          'Obesity_Type_II': 5,
                          'Obesity_Type_III': 6,}
                         }
data_preprocess.replace(dic_to_replace, inplace=True)

In [16]:
target_name = 'BMI'
labels = data_preprocess[target_name]
features = data_preprocess.drop(target_name, axis=1)

In [18]:
from sklearn.preprocessing import MinMaxScaler
features = MinMaxScaler().fit_transform(features)

In [19]:
from sklearn.model_selection import train_test_split

train_features, test_features, train_labels, test_labels = train_test_split(
    features, labels, train_size=0.8, random_state=123
)

## RandomForest Classifier 

In [20]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier()

rfc.fit(train_features,train_labels)

RandomForestClassifier()

In [22]:
train_score = rfc.score(train_features, train_labels)
test_score = rfc.score(test_features, test_labels)

print(f"Classical Random Forest Classifier on the training dataset: {train_score:.2f}")
print(f"Classical Random Forest Classifier on the test dataset: {test_score:.2f}")

Classical Random Forest Classifier on the training dataset: 1.00
Classical Random Forest Classifier on the test dataset: 0.93


## Logistic Regression 

In [23]:
lr=LogisticRegression()
lr.fit(train_features,train_labels)

C:\Users\Admin\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [24]:
train_score = lr.score(train_features, train_labels)
test_score = lr.score(test_features, test_labels)

print(f"Classical Logistic Regression on the training dataset: {train_score:.2f}")
print(f"Classical Logistic Regression on the test dataset: {test_score:.2f}")

Classical Logistic Regression on the training dataset: 0.70
Classical Logistic Regression on the test dataset: 0.69


## SVC(Support vector classifier) 

In [25]:
svc=SVC()
svc.fit(train_features,train_labels)

SVC()

In [26]:
train_score = svc.score(train_features, train_labels)
test_score = svc.score(test_features, test_labels)

print(f"Classical Logistic Regression on the training dataset: {train_score:.2f}")
print(f"Classical Logistic Regression on the test dataset: {test_score:.2f}")

Classical Logistic Regression on the training dataset: 0.74
Classical Logistic Regression on the test dataset: 0.74


## KNN 

In [28]:
knn=KNeighborsClassifier()
knn.fit(train_features,train_labels)

KNeighborsClassifier()

In [29]:
train_score = knn.score(train_features, train_labels)
test_score = knn.score(test_features, test_labels)

print(f"Classical Logistic Regression on the training dataset: {train_score:.2f}")
print(f"Classical Logistic Regression on the test dataset: {test_score:.2f}")

Classical Logistic Regression on the training dataset: 0.82
Classical Logistic Regression on the test dataset: 0.79


## Decision tree

In [31]:
dt=DecisionTreeClassifier()
dt.fit(train_features,train_labels)

DecisionTreeClassifier()

In [32]:
train_score = dt.score(train_features, train_labels)
test_score = dt.score(test_features, test_labels)

print(f"Classical Logistic Regression on the training dataset: {train_score:.2f}")
print(f"Classical Logistic Regression on the test dataset: {test_score:.2f}")

Classical Logistic Regression on the training dataset: 1.00
Classical Logistic Regression on the test dataset: 0.92
